In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

d:\Anaconda3\envs\LLM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.0,
    model_name="gpt-3.5-turbo",
)

d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Load up your HTML from your company page

In [12]:
def get_company_page(company_path):
    y_combinator_url = f"https://www.ycombinator.com{company_path}"
    
    print (y_combinator_url)

    loader = UnstructuredURLLoader(urls=[y_combinator_url])
    return loader.load()


# Get the data of the company you're interested in
data = get_company_page("/companies/Gitlab")

print (f"You have {len(data)} document(s)")
print (f"Preview of your data:\n\n{data[0].page_content[:30]}")


# Split up the texts so you don't run into token limits
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 0
)
docs = text_splitter.split_documents(data)
print (f"You now have {len(docs)} documents")


https://www.ycombinator.com/companies/Gitlab
You have 1 document(s)
Preview of your data:

404

File Not Found

Back to t
You now have 1 documents


### Write your custom prompt templates
These will be used for your specific use case:

1. `map_prompt` will be the prompt that is done on your first pass of your documents
2. `combine_prompt` will be the prompt that is used when you combine the outputs of your map pass

In [4]:
map_prompt = """Below is a section of a website about {prospect}

Write a concise summary about {prospect}. If the information is not about {prospect}, exclude it from your summary.

{text}

% CONCISE SUMMARY:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text", "prospect"])

In [5]:
combine_prompt = """
Your goal is to write a personalized outbound email from {sales_rep}, a sales rep at {company} to {prospect}.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

% INFORMATION ABOUT {company}:
{company_information}

% INFORMATION ABOUT {prospect}:
{text}

% INCLUDE THE FOLLOWING PIECES IN YOUR RESPONSE:
- Start the email with the sentence: "We love that {prospect} helps teams..." then insert what they help teams do.
- The sentence: "We can help you do XYZ by ABC" Replace XYZ with what {prospect} does and ABC with what {company} does 
- A 1-2 sentence description about {company}, be brief
- End your email with a call-to-action such as asking them to set up time to talk more

% YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["sales_rep", "company", "prospect", \
                                                                         "text", "company_information"])

In [6]:
company_information = """
* RapidRoad helps product teams build product faster
* We have a platform that allows product teams to talk more, exchange ideas, and listen to more customers
* Automated project tracking: RapidRoad could use machine learning algorithms to automatically track project progress, identify potential bottlenecks, and suggest ways to optimize workflows. This could help product teams stay on track and deliver faster results.
* Collaboration tools: RapidRoad could offer built-in collaboration tools, such as shared task lists, real-time messaging, and team calendars. This would make it easier for teams to communicate and work together, even if they are in different locations or time zones.
* Agile methodology support: RapidRoad could be specifically designed to support agile development methodologies, such as Scrum or Kanban. This could include features like sprint planning, backlog management, and burndown charts, which would help teams stay organized and focused on their goals.
"""

### LangChain Magic

In [8]:
llm = OpenAI(temperature=.7)

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt_template,
                             combine_prompt=combine_prompt_template,
                             verbose=True
                            )

In [13]:
output = chain({"input_documents": docs, # The seven docs that were created before
                "company": "RapidRoad", \
                "company_information" : company_information,
                "sales_rep" : "Greg", \
                "prospect" : "Gitlab"
               })



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is a section of a website about Gitlab

Write a concise summary about Gitlab. If the information is not about Gitlab, exclude it from your summary.

404

File Not Found

Back to the homepage

For support please contact software@ycombinator.com

% CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Your goal is to write a personalized outbound email from Greg, a sales rep at RapidRoad to Gitlab.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

% INFORMATION ABOUT RapidRoad:

* RapidRoad helps product teams build product faster
* We have a platform that allows product teams to talk more, exchange id

In [14]:
print (output['output_text'])


Dear GitLab Team,

We love that GitLab helps teams develop, deploy, and operate software quickly and efficiently. We believe that RapidRoad can help make the development process even faster and more productive.

RapidRoad is a platform that helps product teams build products faster. We have automated project tracking, collaboration tools, and agile methodology support that can help teams stay organized and focused on their goals.

We can help you develop, deploy, and operate software quickly and efficiently by providing automated project tracking, collaboration tools, and agile methodology support. We believe this can help you deliver faster results and achieve your product goals.

We'd love to chat more about how our platform can help you and your team. Please let me know if you have a few minutes to discuss further.

Best,

Greg
RapidRoad Sales Rep


### Rinse and Repeat: Loop Through Companies

In [20]:
import pandas as pd

In [16]:
df_companies = pd.read_clipboard()

In [19]:
df_companies

,pip,install,python-magic-bin==0.4.14


In [18]:
for i, company in df_companies.iterrows():
    print (f"{i + 1}. {company['Name']}")
    page_data = get_company_page(company['Link'])
    docs = text_splitter.split_documents(page_data)

    output = chain({"input_documents": docs, \
                "company":"RapidRoad", \
                "sales_rep" : "Greg", \
                "prospect" : company['Name'],
                "company_information" : company_information
               })
    
    print (output['output_text'])
    print ("\n\n")